In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = '../data/ESP_PUBLIC.IDENTITE_ARBRE.csv'
df = pd.read_csv(file_path, sep=',', header = 0, index_col=False)

In [4]:
# df.head(50)

In [7]:
coloumn_interest 
list(df.columns[:])

['ELEM_POINT_ID',
 'CODE',
 'NOM',
 'GENRE',
 'GENRE_DESC',
 'CATEGORIE',
 'CATEGORIE_DESC',
 'SOUS_CATEGORIE',
 'SOUS_CATEGORIE_DESC',
 'CODE_PARENT',
 'CODE_PARENT_DESC',
 'ADR_SECTEUR',
 'BIEN_REFERENCE',
 'GENRE_BOTA',
 'ESPECE',
 'VARIETE',
 'STADEDEDEVELOPPEMENT',
 'EQUIPE',
 'REMARQUES',
 'ANNEEDEPLANTATION',
 'RAISONDEPLANTATION',
 'TRAITEMENTCHENILLES',
 'COURRIER',
 'IDENTIFIANTPLU',
 'TYPEIMPLANTATIONPLU',
 'INTITULEPROTECTIONPLU',
 'ANNEEABATTAGE',
 'ESSOUCHEMENT',
 'DIAMETREARBRE',
 'CAUSEABATTAGE',
 'COLLECTIVITE',
 'GeoJSON']